# NAMD Protein-ligand complex MD Setup tutorial using Jupyter Notebook
This notebook is wrritten by quantaosun@gmail.com in Shanghai, China, 2021. This notebook took PDB ID 7L10, the protease of SARS-CoV-2 with an inhibitor, as an example, relative work was firstly published on ACS Cent. Sci. 2021, 7, 3, 467–475, by William L. Jorgensen, Yale University.

Ambertools (https://ambermd.org/AmberTools.php) and Openmm (https://openmm.org/)should be properly refer to if any simulation result would be used in any publications.

In [30]:
#install openmm
!yes | conda install -c conda-forge openmm &> /dev/null
# install openmm-setup for protein fix and simulation configuration
!yes | conda install -c conda-forge openmm-setup 
#install open babel
!yes|conda install -c openbabel openbabel &> /dev/null
# install Amber tools
!yes|conda install -c conda-forge ambertools=21 compilers 
#install visualization
! pip install py3Dmol 

yes: standard output: Broken pipe
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

yes: standard output: Broken pipe
yes: standard output: Broken pipe
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/q/anaconda3/envs/Amber

  added / updated specs:
    - ambertools=21
    - compilers


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2021.10.26~ --> conda-forge::ca-certificates-2021.10.8-ha878542_0


Proceed ([y]/n)? 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
yes: standard output: Broken 

In [31]:
import sys
sys.path.append('/home/sqt/anaconda3/envs/Amber/lib/python3.7/site-packages')

# Download your PDB

In [16]:
PDB_ID = "7L10" #@param {type:"string"}
pdb = PDB_ID + ".pdb"
! wget https://files.rcsb.org/download/$pdb

--2021-12-02 20:23:52--  https://files.rcsb.org/download/7L10.pdb
Resolving files.rcsb.org (files.rcsb.org)... 132.249.210.134
Connecting to files.rcsb.org (files.rcsb.org)|132.249.210.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘7L10.pdb.1’

7L10.pdb.1              [          <=>       ] 241.42K  13.5KB/s    in 17s     

2021-12-02 20:24:12 (14.2 KB/s) - ‘7L10.pdb.1’ saved [247212]



# Fix your protein with PDBfixer implemented in openmm-setup, please save the final strcture as "output.pdb" !!!

In [12]:
!openmm-setup
#Click the square pause button to force quit after finished.

 * Serving Flask app 'openmmsetup.openmmsetup' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Dec/2021 19:02:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 19:02:55] "GET /static/Icon.svg HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 19:02:55] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/Dec/2021 19:03:08] "GET /selectFiles?type=pdb HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 19:03:08] "GET /static/Icon.svg HTTP/1.1" 304 -
127.0.0.1 - - [02/Dec/2021 19:03:28] "POST /configureFiles HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 19:03:28] "GET /static/ngl.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 19:03:28] "GET /static/Icon.svg HTTP/1.1" 304 -
127.0.0.1 - - [02/Dec/2021 19:03:28] "GET /getCurrentStructure HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 19:03:37] "POST /selectChains HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 19:03:37] "GET /static/Icon.

# Go to H++, register and process the output.pdb structure, download top and crd. If you leave everything default, you don't need to change anything below.http://biophysics.cs.vt.edu/index.php

# It is assumed you have save the processed pdb with a name of "output.pdb", otherwise please modifiy commands below.

# Generate protein pdb

In [17]:
!ambpdb -p 0.15_80_10_pH6.5_output.top -c 0.15_80_10_pH6.5_output.crd > 0.pdb

In [18]:
!awk '$4=="XEY"' 7L10.pdb > ligand1.pdb 

# Add hydrogens to you ligand

In [19]:
!obabel -ipdb ligand1.pdb -opdb -O ligand2.pdb -h

1 molecule converted


# Beautify pdb format

In [20]:
!pdb4amber -i ligand2.pdb -o ligand3.pdb


Summary of pdb4amber for: ligand2.pdb

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
None
-----------Non-standard-resnames
XEY

---------- Mising heavy atom(s)

None


# Generate ligand mol2 file

In [21]:
!antechamber -fi pdb -i ligand3.pdb -fo mol2 -o ligand4.mol2 -c bcc -pf y


Welcome to antechamber 21.0: molecular input file processor.

acdoctor mode is on: check and diagnose problems in the input file.
The atom type is set to gaff; the options available to the -at flag are
    gaff, gaff2, amber, bcc, and sybyl.
-- Check Format for pdb File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

/bin/bash: /home/q/anaconda3/envs/Amber/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/q/anaconda3/envs/Amber/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Info: Total number of electrons: 214; net charge: 0

Running: /home/q/anaconda3/envs/Amber/bin/sqm -O -i sqm.in -o sqm.out
/bin/bash: /home/q/anaconda3/envs/Am

# Make sure you see "calculation completed", otherwise something may have been wrong

In [24]:
!tail  sqm.out

  QMMM:    36       36      H        6.5586   -1.5201   19.1778
  QMMM:    37       37      H       11.5122    5.3225   20.3572
  QMMM:    38       38      H       13.0977    5.3892   22.2814
  QMMM:    39       39      H       13.1130    3.5286   23.9487
  QMMM:    40       40      H       11.5593    1.6013   23.6837
  QMMM:    41       41      H        5.4003   -1.8655   27.1377
  QMMM:    42       42      H       10.1644    0.8405   19.6508

           --------- Calculation Completed ----------



# Generate ligand prepi file

In [25]:
!antechamber -i ligand4.mol2 -fi mol2 -o ligand5.prepi -fo prepi -pf y


Welcome to antechamber 21.0: molecular input file processor.

acdoctor mode is on: check and diagnose problems in the input file.
The atom type is set to gaff; the options available to the -at flag are
    gaff, gaff2, amber, bcc, and sybyl.
-- Check Format for mol2 File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

/bin/bash: /home/q/anaconda3/envs/Amber/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/q/anaconda3/envs/Amber/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/q/anaconda3/envs/Amber/lib/libtinfo.so.6: no version information available (required by /bin/bash)



# Generate ligand frcmod file

In [26]:
!parmchk2 -f prepi -i ligand5.prepi -o ligand6.frcmod

# Combine protein with ligand again

In [27]:
!cat 0.pdb ligand3.pdb > complex.pdb

# (Not working properly for now) Beautify PDB format, but we might just not need this anyway, skip for now.

In [28]:
!pdb4amber -i complex.pdb -o complex_clean.pdb


Summary of pdb4amber for: complex.pdb
Traceback (most recent call last):
  File "/home/q/anaconda3/envs/Amber/bin/pdb4amber", line 33, in <module>
    sys.exit(load_entry_point('pdb4amber==20.1', 'console_scripts', 'pdb4amber')())
  File "/home/q/anaconda3/envs/Amber/lib/python3.7/site-packages/pdb4amber/pdb4amber.py", line 835, in main
    arg_noter=opt.noter)
  File "/home/q/anaconda3/envs/Amber/lib/python3.7/site-packages/pdb4amber/pdb4amber.py", line 520, in run
    pdbfixer._write_renum(base_filename)
  File "/home/q/anaconda3/envs/Amber/lib/python3.7/site-packages/pdb4amber/pdb4amber.py", line 390, in _write_renum
    residue.idx + 1))
TypeError: %c requires int or char


# Create and run a file called tleap.in with Ambertools

In [29]:
com_file = open('tleap.in','w')
com_file.write('''
source leaprc.protein.ff14SB #Source leaprc file for ff14SB protein force field
source leaprc.gaff #Source leaprc file for gaff
source leaprc.water.tip3p #Source leaprc file for TIP3P water model
loadamberprep ligand5.prepi #Load the prepi file for the ligand
loadamberparams ligand6.frcmod #Load the additional frcmod file for ligand
mol = loadpdb complex.pdb #Load PDB file for protein-ligand complex
solvatebox mol TIP3PBOX 8 #Solvate the complex with a cubic water box
addions mol Cl- 0 #Add Cl- ions to neutralize the system
saveamberparm mol complex.prmtop complex.inpcrd #Save AMBER topology and coordinate files
quit #Quit tleap program
''')
com_file.close()
!tleap -s -f tleap.in > tleap.out

# Upload prmtop and inpcrd to openmm-setup, and start the simulation! If you have a proper GPU, select CUDA, otherwise select CPU.

In [ ]:
!openmm-setup
#Click the square pause button to force quit after finished.

 * Serving Flask app 'openmmsetup.openmmsetup' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Dec/2021 20:40:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:40:47] "GET /static/Icon.svg HTTP/1.1" 304 -
127.0.0.1 - - [02/Dec/2021 20:40:52] "GET /selectFiles?type=amber HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:40:52] "GET /static/Icon.svg HTTP/1.1" 304 -
127.0.0.1 - - [02/Dec/2021 20:41:11] "POST /configureFiles HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:41:12] "GET /static/Icon.svg HTTP/1.1" 304 -
127.0.0.1 - - [02/Dec/2021 20:41:12] "POST /setSimulationOptions HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:41:22] "POST /setSimulationOptions HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:41:41] "GET /showRunSimulation HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:41:41] "GET /static/Icon.svg HTTP/1.1" 304 -
127.0.0.1 - - [02/Dec/2021 20:42:02

127.0.0.1 - - [02/Dec/2021 20:45:02] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:45:04] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:45:06] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:45:08] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:45:10] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:45:12] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:45:14] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:45:16] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:45:18] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:45:20] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:45:22] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:45:24] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:45:26] "GET /getSimula

127.0.0.1 - - [02/Dec/2021 20:48:30] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:48:32] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:48:34] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:48:36] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:48:38] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:48:40] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:48:42] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:48:44] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:48:46] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:48:48] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:48:50] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:48:52] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:48:54] "GET /getSimula

127.0.0.1 - - [02/Dec/2021 20:51:58] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:52:00] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:52:02] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:52:04] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:52:06] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:52:08] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:52:10] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:52:12] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:52:14] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:52:16] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:52:18] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:52:20] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:52:22] "GET /getSimula

127.0.0.1 - - [02/Dec/2021 20:55:26] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:55:28] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:55:30] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:55:32] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:55:34] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:55:36] "GET /getSimulationOutput HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:55:36] "POST /stopSimulation HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 20:55:38] "GET /getSimulationOutput HTTP/1.1" 404 -


# Alternatively, you could save the simulation input files and run it on a cloud or HPC based OpenMM.

# If all goes well, you should see something like below (CPU only).

In [ ]:
Building system...
Performing energy minimization...
Equilibrating...
Simulating...
#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"
0.1%	1000	-488259.139341292	298.57624202824957	0
0.2%	2000	-492082.04384769656	300.09275305741824	16.6
0.3%	3000	-494725.1462343714	302.3090730320394	16.6
0.4%	4000	-496075.06792005355	302.3939891905172	16.4
0.5%	5000	-498015.23323377833	300.135566206444	16.4
0.6%	6000	-498488.50188332796	298.5545999644654	16.3
0.7%	7000	-499572.9765659773	300.7757759109986	16.3
0.8%	8000	-499030.79245316726	297.2492234858693	16.2
0.9%	9000	-499823.2656625096	300.60293949805	16.2
1.0%	10000	-500134.98822511156	297.5748675220697	16.2
1.1%	11000	-499849.7045947542	300.5532217535929	16.2
1.2%	12000	-500542.2799043103	299.2530600741315	16.2